In [3]:
import json
import pandas as pd
import numpy as np

In [ ]:
with open("oxford_sensewords_original.json", "r") as f:
    oxford_sensewords = json.load(f)

In [4]:
with open("oxford_sensewords_meta.json", "r") as f:
    oxford_sensewords_meta = json.load(f)

In [ ]:
len(list(oxford_sensewords.keys())), len(list(oxford_sensewords_meta.keys()))

In [ ]:
overall_words = 0 
empty_count = 0
for key in oxford_sensewords.keys():
    if len(oxford_sensewords[key]) == 0: empty_count += 1
    overall_words += len(oxford_sensewords[key])
overall_words, empty_count

In [ ]:
overall_words = 0 
empty_count = 0
for key in oxford_sensewords_meta.keys():
    if len(oxford_sensewords_meta[key]) == 0: empty_count += 1
    overall_words += len(oxford_sensewords_meta[key])
overall_words, empty_count

In [ ]:
oxford_sensewords['word']

In [5]:
oxford_sensewords_meta['word']

['mot', 'bruit [masculine] qui court', 'parole', 'formuler', 'nouvelles']

In [26]:
df = pd.read_csv("./lookup results/naive_transaltion_en_ru_fr_LEMMAS_WHOLE_LIST.csv")
df.head()

,Unnamed: 0,word,local_word,examples,local_examples
0,0,for,pour,"['for the bank', 'for last month', 'for one ye...","['pour la banque', 'pour le mois dernier', 'pe..."
1,1,for,pour,['for god'],['pour dieu']
2,2,for,au cours de,[],[]
3,3,for,à l'égard de,[],[]
4,4,for,à des fins,[],[]


In [6]:
def csv_to_dict(df):
    senseword_dict = {}
    for ind in range(len(df)):
        df_el = list(df.iloc[ind])
        #print(list(df.iloc[ind]))
        #print(df_el[1],df_el[2])
        if df_el[1] in senseword_dict:
            senseword_dict[df_el[1].lower()].append(df_el[2].lower())
        else:
            senseword_dict[df_el[1].lower()] = []
            senseword_dict[df_el[1].lower()].append(df_el[2].lower())
    return senseword_dict

In [27]:
yandex_naive_sensewords = {}
for ind in range(len(df)):
    df_el = list(df.iloc[ind])
    #print(list(df.iloc[ind]))
    #print(df_el[1],df_el[2])
    if df_el[1] in yandex_naive_sensewords:
        yandex_naive_sensewords[df_el[1].lower()].append(df_el[2].lower())
    else:
        yandex_naive_sensewords[df_el[1].lower()] = []
        yandex_naive_sensewords[df_el[1].lower()].append(df_el[2].lower())
    

In [28]:
df_direct = pd.read_csv("./lookup results/yandex_lookup_en_fr_lemmas_only_WHOLE_LIST.csv")
df_direct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 5 columns):
Unnamed: 0        287 non-null int64
word              287 non-null object
local_word        287 non-null object
examples          287 non-null object
local_examples    287 non-null object
dtypes: int64(1), object(4)
memory usage: 11.3+ KB


In [29]:
yandex_direct_lookup_sensewords = {}
for ind in range(len(df_direct)):
    df_el = list(df_direct.iloc[ind])
    #print(list(df.iloc[ind]))
    #print(df_el[1],df_el[2])
    if df_el[1] in yandex_direct_lookup_sensewords:
        yandex_direct_lookup_sensewords[df_el[1].lower()].append(df_el[2].lower())
    else:
        yandex_direct_lookup_sensewords[df_el[1].lower()] = []
        yandex_direct_lookup_sensewords[df_el[1].lower()].append(df_el[2].lower())
    

In [ ]:
yandex_direct_lookup_sensewords['nobody']

In [ ]:
yandex_naive_sensewords['nobody']

In [ ]:
oxford_sensewords_meta['nobody']

In [ ]:
a = set([1,2,3,4])
b = set([2,3])
a.intersection(b)

In [21]:
def get_intersection(oxford_senses, compared_senses):
    detected_words_oxford = []
    detcted_words_compared = []
    for ox_word in oxford_senses:
        for comp_word in compared_senses:
            if ox_word == comp_word or ox_word in comp_word or comp_word in ox_word:
                detected_words_oxford.append(ox_word)
                detcted_words_compared.append(comp_word)
    oxford_set = set(oxford_senses)
    compared_set = set(compared_senses)
    intersect_oxford = oxford_set.intersection(set(detected_words_oxford))
    if (len(set(oxford_senses))) == 0:
        oxford_senses = ['dummy_word']
    oxford_recall = len(intersect_oxford) / (len(set(oxford_senses)))
    intersect_compared = compared_set.intersection(set(detcted_words_compared))
    new_words_from_compare = (len(compared_set) - len(intersect_compared)) / (len(set(oxford_senses)))
    print(intersect_oxford, intersect_compared,oxford_recall,new_words_from_compare, (oxford_recall + new_words_from_compare)/2  )
    return (oxford_recall + new_words_from_compare)/2

#get_intersection(oxford_sensewords['word'] ,yandex_naive_sensewords['word'])


In [31]:
def sense_words_dicts_comparison(oxford, comparison_dict):
    word = []
    oxford_sensewords = []
    compared_sensewords = []
    comparison_value = []
    error_count = 0
    error_list = []
    for ox_key in oxford.keys(): 
        try:
            word.append(ox_key)
            oxford_sensewords.append(oxford[ox_key])
            compared_sensewords.append(comparison_dict[ox_key.lower()])
            print (ox_key, "\noxford sensewords", oxford[ox_key],"\ncompared sensewords",comparison_dict[ox_key.lower()])
            comp_res = get_intersection(oxford[ox_key],comparison_dict[ox_key.lower()])
            print()
            comparison_value.append(comp_res)
        except:
            error_count += 1
            error_list.append(ox_key)
    print(error_count, "errors occured", error_list)
    data = pd.DataFrame(list(zip(word,oxford_sensewords,compared_sensewords, comparison_value)),columns =['word', 'oxford_sensewords', 'compared_sensewords','comparison_value'])
    return data    

# naive translation from rus lookup

In [32]:
comp_df = sense_words_dicts_comparison(oxford_sensewords_meta ,yandex_naive_sensewords)        


word 
oxford sensewords ['mot', 'bruit [masculine] qui court', 'parole', 'formuler', 'nouvelles'] 
compared sensewords ['mot', 'nouvelle', 'mot', 'dire', 'formuler', 'textuel', 'verbal']
{'mot', 'formuler', 'nouvelles'} {'mot', 'formuler', 'nouvelle'} 0.6 0.6 0.6

the 
oxford sensewords ['les', 'la (before a consonant) / l’…', 'le (before a consonant) / l’ (before a vowel)', 'la (before a consonant) / l’ (before a vowel)'] 
compared sensewords ['ce', 'le', 'le', 'tant']
{'les', 'le (before a consonant) / l’ (before a vowel)'} {'le'} 0.5 0.5 0.5

and 
oxford sensewords ['(et)', '(indique une intention ou un but)', 'plus', 'et', '(et même)', 'de'] 
compared sensewords ['et', 'et aussi', 'et en outre', 'donc et', 'avec']
{'(et)', 'et', '(et même)'} {'et en outre', 'et', 'et aussi', 'donc et'} 0.5 0.16666666666666666 0.3333333333333333

that 
oxford sensewords ['que (before a consonant) / qu’ (before a vowel)', 'cette', 'cette … (là)', 'comme…', 'celui/celle-là', 'qui', 'ce (before a conso

In [ ]:
comp_df.head()

In [ ]:
comp_df.to_csv("yandexNaive_vs_cambridge_comparison_WHOLE_LIST.csv")

In [33]:
np.mean(list(comp_df['comparison_value'])), np.median(list(comp_df['comparison_value']))

(0.9171662376235585, 0.6666666666666666)

# direct lookup

In [34]:
comp_df_direct = sense_words_dicts_comparison(oxford_sensewords_meta ,yandex_direct_lookup_sensewords)        

word 
oxford sensewords ['mot', 'bruit [masculine] qui court', 'parole', 'formuler', 'nouvelles'] 
compared sensewords ['mot']
{'mot'} {'mot'} 0.2 0.0 0.1

the 
oxford sensewords ['les', 'la (before a consonant) / l’…', 'le (before a consonant) / l’ (before a vowel)', 'la (before a consonant) / l’ (before a vowel)'] 
compared sensewords ['le', 'au']
{'les', 'le (before a consonant) / l’ (before a vowel)'} {'le'} 0.5 0.25 0.375

and 
oxford sensewords ['(et)', '(indique une intention ou un but)', 'plus', 'et', '(et même)', 'de'] 
compared sensewords ['et']
{'(et)', 'et', '(et même)'} {'et'} 0.5 0.0 0.25

that 
oxford sensewords ['que (before a consonant) / qu’ (before a vowel)', 'cette', 'cette … (là)', 'comme…', 'celui/celle-là', 'qui', 'ce (before a consonant) / cet (before a vowel)', 'ce (before a consonant) / c’ (before a vowel)', 'cela / ça', 'si', 'tellement', 'ce (before a consonant) / cet (before a vowel) … (là)', 'que'] 
compared sensewords ['que', 'qui', 'ce']
{'cette … (là)',

compared sensewords ['fichier', 'déposer']
{'fichier'} {'fichier'} 0.08333333333333333 0.08333333333333333 0.08333333333333333

fill 
oxford sensewords ['répondre à', '(se) remplir', 'prendre', 'à sa faim', 'boucher', 'pourvoir', '(r)emplir', 'plomber', 'remplir'] 
compared sensewords ['remplir', 'remplissage']
{'remplir', '(se) remplir'} {'remplir'} 0.2222222222222222 0.1111111111111111 0.16666666666666666

final 
oxford sensewords ['finale', 'examens [masculine', 'dernier/-ière', 'final', 'final/-ale', 'plural] de fin d’année', 'définitif', 'définitif/-ive', 'dernier'] 
compared sensewords ['final', 'finale']
{'final', 'finale', 'final/-ale'} {'final', 'finale'} 0.3333333333333333 0.0 0.16666666666666666

finally 
oxford sensewords ['finalement', 'enfin', 'pour terminer'] 
compared sensewords ['enfin']
{'enfin'} {'enfin'} 0.3333333333333333 0.0 0.16666666666666666

finger 
oxford sensewords ['toucher', 'langue', 'doigt'] 
compared sensewords ['doigt']
{'doigt'} {'doigt'} 0.3333333333

In [35]:
np.mean(list(comp_df_direct['comparison_value'])), np.median(list(comp_df_direct['comparison_value']))

(0.31932067683859006, 0.25)

# DIRECT LOOKUP SYSTRAN

In [8]:
df_systran_en_fr = pd.read_csv("systran_en_fr.csv")
df_systran_en_fr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326 entries, 0 to 1325
Data columns (total 5 columns):
Unnamed: 0        1326 non-null int64
word              1326 non-null object
local_word        1326 non-null object
examples          1326 non-null object
local_examples    1326 non-null object
dtypes: int64(1), object(4)
memory usage: 51.9+ KB


In [10]:
systran_words = csv_to_dict(df_systran_en_fr)
systran_words['word']

['mot',
 'parole',
 'terme',
 'propos',
 'mention',
 'expression',
 'discours',
 'formuler',
 'libeller']

In [22]:
comp_df_systran_direct = sense_words_dicts_comparison(oxford_sensewords_meta ,systran_words)        

word oxford sensewords ['mot', 'bruit [masculine] qui court', 'parole', 'formuler', 'nouvelles'] compared sensewords ['mot', 'parole', 'terme', 'propos', 'mention', 'expression', 'discours', 'formuler', 'libeller']
{'mot', 'parole', 'formuler'} {'mot', 'parole', 'formuler'} 0.6 1.2 0.8999999999999999

the oxford sensewords ['les', 'la (before a consonant) / l’…', 'le (before a consonant) / l’ (before a vowel)', 'la (before a consonant) / l’ (before a vowel)'] compared sensewords ['le']
{'les', 'le (before a consonant) / l’ (before a vowel)'} {'le'} 0.5 0.0 0.25

and oxford sensewords ['(et)', '(indique une intention ou un but)', 'plus', 'et', '(et même)', 'de'] compared sensewords ['et']
{'(et)', 'et', '(et même)'} {'et'} 0.5 0.0 0.25

that oxford sensewords ['que (before a consonant) / qu’ (before a vowel)', 'cette', 'cette … (là)', 'comme…', 'celui/celle-là', 'qui', 'ce (before a consonant) / cet (before a vowel)', 'ce (before a consonant) / c’ (before a vowel)', 'cela / ça', 'si', '

In [23]:
comp_df_systran_direct.to_csv("systran_direct_vs_cambridge_comparison.csv")

In [20]:
comp_df_systran_direct.head(30)

,word,oxford_sensewords,compared_sensewords,comparison_value
0,word,"[mot, bruit [masculine] qui court, parole, for...","[mot, parole, terme, propos, mention, expressi...",0.900000
1,the,"[les, la (before a consonant) / l’…, le (befor...",[le],0.250000
2,and,"[(et), (indique une intention ou un but), plus...",[et],0.250000
3,that,[que (before a consonant) / qu’ (before a vowe...,"[que, ce, ce... -là, cela / ça / ce]",0.384615
4,for,"[en, en faveur de, malgré, pour, en raison de,...","[car, pour, de, à, dans]",0.437500
5,with,"[(avec), contre, pour, en + verbe au participe...","[avec, à, par]",0.277778
6,but,"[mais, excepté, sauf]","[mais, sauf, à l'exception de]",0.500000
7,you,"[tu/toi/te, espèce de, on, tu/vous, vous]","[tu, vous, vous, te, toi / vous]",0.300000
8,this,"[cette, ce (before a consonant) / cet (before ...","[ce, ce... -ci / celui-ci, ce / ceci]",0.500000
9,not,"[ne…pas, pas, non, ne … pas, n’est-ce pas]","[pas, non]",0.500000


In [16]:
np.mean(list(comp_df_systran_direct['comparison_value'])), np.median(list(comp_df_systran_direct['comparison_value']))

(0.6738429835296843, 0.5625)